In [1]:
import os
import random
from argparse import ArgumentParser
import logging

import torch
from trl import SFTConfig, SFTTrainer

from lima_dataset import load_lima_dataset, tokenize_text, format_prompt_func, EOT_TOKEN
from utils import (
    read_yaml,
    get_model_config,
    get_tokenizer_config,
    get_split_config,
    get_dataset_config,
    get_trainer_config,
    get_generation_config,
    get_generation_samples,
    get_lora_config,
    _handle_seed,
    DEVICE,
)
from model import (
    tokenize_text,
    load_model,
    load_tokenizer,
    load_lora_model,
    generate,
    compute_metrics,
)

In [2]:
from transformers import LlamaTokenizer

In [3]:
# tokenizer_temp = LlamaTokenizer.from_pretrained(
#     "meta-llama/Llama-2-7b-hf", add_bos_token=True, add_eos_token=True
# )
# outs = tokenizer_temp.encode("hi i am harsh")
# outs
# tokenizer_temp.build_inputs_with_special_tokens([1, 7251, 474, 626, 4023, 845])d

In [4]:
config = read_yaml("./configs/train_config_llama_lora.yaml")

In [5]:
tokenizer_name, tokenizer_path, tokenizer_config = get_tokenizer_config(config)
tokenizer = load_tokenizer(
    tokenizer_name=tokenizer_name,
    tokenizer_path=tokenizer_path,
    tokenizer_config=tokenizer_config,
)
tokenizer_name, tokenizer_path, tokenizer_config

('llama2',
 'meta-llama/Llama-2-7b-hf',
 {'add_bos_token': True, 'add_eos_token': True})

In [6]:
tokenizer.add_eos_token

True

In [7]:
model_name, model_path, base_model_path, model_config = get_model_config(config)
model = load_model(
    model_string=model_name,
    model_path=model_path,
    base_model_path=base_model_path,
    model_config=model_config,
)
model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32002, 4096)

In [9]:
lora_config = get_lora_config(config)
model = load_lora_model(model, lora_config)

In [10]:
dataset_desc, (train_split_config, val_split_config, test_split_config) = (
    get_split_config(config)
)
dataset_desc, train_split_config

('LIMA Instruct Finetunning Dataset',
 {'dataset_path': 'GAIR/lima', 'sub_split_size': None})

In [11]:
train_dataset_path, train_sub_split_size, train_dataset_config = get_dataset_config(
    train_split_config
)
train_dataset_path, train_sub_split_size, train_dataset_config

('GAIR/lima', None, {})

In [12]:
train_dataset = load_lima_dataset(
    train_dataset_path, "train", train_sub_split_size, **train_dataset_config
)

In [ ]:
trainer_config = get_trainer_config(config)
trainer_config["logging_dir"] = os.path.join(
    trainer_config["output_dir"], "runs", trainer_config["run_name"]
)
save_trained_model = trainer_config.pop("save_trained_model", True)
sft_trainer_args = SFTConfig(**trainer_config)

In [ ]:
save_trained_model = trainer_config.pop("save_trained_model", True)
resume_from_checkpoint = trainer_config.pop("resume_from_checkpoint", None)
sft_trainer_args = SFTConfig(**trainer_config)

sft_trainer = SFTTrainer(
    model,
    args=sft_trainer_args,
    train_dataset=train_dataset,
    formatting_func=format_prompt_func,
    processing_class=tokenizer,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
tokenizer

LlamaTokenizer(name_or_path='meta-llama/Llama-2-7b-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '[pad]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("[pad]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("[EOT]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}
)

In [11]:
sft_training_outs = sft_trainer.train(resume_from_checkpoint=resume_from_checkpoint)

KeyboardInterrupt: 

In [12]:
prompt = f'{train_dataset[0]["conversations"][0]}'
prompt

'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).'

In [13]:
generation_config = get_generation_config(config)
generation_config

In [ ]:
prompt_temp = f"{prompt} {EOT_TOKEN}"
tokenized_prompts = tokenize_text(
    prompt_temp, tokenizer, use_encode=True, return_tensors="pt"
)

In [47]:
from transformers import GenerationConfig

In [58]:
model.config.bos_token_id, model.config.eos_token_id, model.config.pad_token_id

(1, 2, None)

In [ ]:
# gen_config_obj = GenerationConfig(**gen_config_temp)
# gen_config_obj.bos_token_id = 

GenerationConfig {
  "max_length": 2048
}

In [46]:
generated_tokens = model.generate(tokenized_prompts, generation_config)
generated_tokens.shape

AttributeError: 'dict' object has no attribute 'bos_token_id'

In [45]:
tokenized_prompts

tensor([[    1, 14350,   385,  4876,   304,   590, 18385, 17088, 10554,   292,
         12251,  6721,   363,  1371,   373,   263,  3271,  1287, 12827,   373,
          4327,   414, 29889, 26321, 29892,   306,  1016, 29915, 29873,  2274,
           278,  4328,  1546,  2094,   397,   414,   322,  1602,   397,   414,
         29889, 29871, 32000]])

In [35]:
from model import generate

In [36]:
outs = generate(
    model,
    prompt_samples=prompt,
    tokenizer=tokenizer,
    generation_config=generation_config,
    use_encode=True,
)

/home/hmankodi/.conda/envs/DL/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/hmankodi/.conda/envs/DL/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [37]:
print(outs[0])

Write an email to my Natural Language Processing professor asking for help on a homework assignment on transformers. Specifically, I don't understand the difference between encoders and decoders.

My professor has been very helpful in responding to questions that I have had regarding the class material so far. However, he is currently out of town traveling with his family overseas (I believe). And since this was due yesterday evening at midnight EST, it seemed like it might be difficult or impossible for him to get back into contact quickly enough before the deadline. Thus, I am writing this e-mail as a way of getting my question answered while also helping me complete my assigned work in time. 

Please note: This is not meant to plagiarize any part of your class materials. In fact, one could argue that using these resources will make it easier for you to answer my question without having to do much research yourself! That said here are some links where they talk about how transformers

In [33]:
prompt

"Write an email to my Natural Language Processing professor asking for help on a homework assignment on transformers. Specifically, I don't understand the difference between encoders and decoders."

In [48]:
prompt = tokenize_text(
    "What is a LLM?[EOT]", tokenizer, use_encode=True, return_tensors="pt"
).to(device="cuda:0")

In [49]:
from transformers import GenerationConfig

In [50]:
generation_config = GenerationConfig(max_length=300)
generation_config

GenerationConfig {
  "max_length": 300
}

In [51]:
outs = model.generate(prompt, generation_config=generation_config)
outs_text = tokenizer.batch_decode(outs)

/home/hmankodi/.conda/envs/DL/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/hmankodi/.conda/envs/DL/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [52]:
print(outs_text[0])

<s> What is a LLM? [EOT] a Master of Laws? A a Master? with "









PA
MS
MSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSPAMSPAMSPAMSMSMSPAMSPAMSPAMSPAMSPAMSPAMSPAMSPAMSMSMSPAMSMSPAMSPAMSPAMSMSMSMSMSPAMSMSMSMSMSPAPAPAMSMSMSPAMSPAMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMSMS
